In [ ]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.envs.safe_metadrive_env import SafeMetaDriveEnv
from metadrive.policy.expert_policy import ExpertPolicy
from metadrive.examples.ppo_expert.torch_expert import torch_expert as expert

from IPython.display import Image, clear_output
import pandas as pd

from pprint import pprint

## Run scenarios


In [ ]:
def run_scenario_n_times_correct(
    n_scenarios=10,
    expert_driving=True,
    use_same_seed=False,
    gif: str = "",
    user_config: dict = {},
) -> list:
    """
    Runs same scenario n time and collects the traces
    """

    traces = []
    config = user_config | {
        "map": 2,
        "num_scenarios": n_scenarios,
    }

    try:

        for rep in range(n_scenarios):

            env = MetaDriveEnv(config=config)
            seed = 0 if use_same_seed else rep

            obs, step_info = env.reset(seed)
            step_info["repetition"] = rep
            traces.append(step_info)
            print(f"{env.current_seed = }")
            while True:

                # get action from expert driving, or a dummy action
                action = (
                    expert(env.agent, deterministic=True)
                    if expert_driving
                    else [0.1, 0.33]
                )
                obs, reward, tm, tr, step_info = env.step(action)
                step_info["repetition"] = rep
                traces.append(step_info)

                if gif:
                    env.render(
                        mode="topdown",
                        # get the overview of the scene
                        film_size=(1000, 1000),
                        screen_size=(1000, 1000),
                        # set camer to map center
                        camera_position=env.current_map.get_center_point(),
                        # auto determine the number of pixels for 1 meter
                        scaling=None,
                        # do not pop window
                        screen_record=True,
                        window=False,
                    )

                if tm or tr:
                    break

            print(f"{gif = }")
            if gif:
                env.top_down_renderer.generate_gif(
                    gif_name=f"gifs/{gif}_seed_{rep}.gif", duration=30
                )

            env.close()
    finally:
        pass

    return traces

In [ ]:
def are_traces_deterministic(traces) -> bool:
    df = pd.DataFrame(traces)

    # grouping by repetition to get a list of traces
    traces = df.groupby("repetition")

    # drop index and repetition ID to compare only step info later
    stripped_traces = [
        trace.reset_index(drop=True).drop("repetition", axis=1) for _, trace in traces
    ]

    # iterate over each trace and check if it is equal to the first one
    are_equal_to_first_trace = [
        trace.equals(stripped_traces[0]) for trace in stripped_traces
    ]

    # if all traces are equal to the first, then all traces are equal hence deterministic
    return all(are_equal_to_first_trace)

## Determinism check


In [ ]:
same_seed = True

traces = run_scenario_n_times_correct(
    n_scenarios=10,
    use_same_seed=same_seed,
)


clear_output()
if same_seed:
    assert are_traces_deterministic(traces), "Traces must be deterministic!"

df = pd.DataFrame(traces)
# df

## Fidelity parameters

### Default settings

- `dt = 0.02`
- `decision_repeat = 5`
- Step time is `dt * decision_repeat = 0.1` which is 10FPS


In [ ]:
# low_fidelity = run_scenario_n_times_correct(
#     n_scenarios=5, user_config={"decision_repeat": 10}, gif=True, use_same_seed=False
# )
clear_output()

In [ ]:
# assert are_traces_deterministic(low_fidelity), "Traces must be deterministic!"
# df = pd.DataFrame(low_fidelity)

In [ ]:
# ===== Termination Scheme =====
termination_sceme = dict(
    out_of_route_done=False,
    on_continuous_line_done=False,
    crash_vehicle_done=True,
    crash_object_done=True,
    crash_human_done=True,
)

In [8]:
import time

df = pd.DataFrame()
times = []
for dr in [5, 10, 15, 20]:
    start = time.perf_counter()
    traces = run_scenario_n_times_correct(
        n_scenarios=1000,
        user_config=termination_sceme | {"decision_repeat": dr, "horizon": 1000},
        # gif=f"dr_{dr}",
        use_same_seed=False,
    )

    traces_df = pd.DataFrame(traces)
    traces_df["decision_repeat"] = dr
    df = pd.concat([df, traces_df])
    times.append((dr, time.perf_counter() - start))
    clear_output()

df.to_csv("data/metadrive.csv")

In [9]:
df.to_csv("data/metadrive.csv")

In [10]:
for dr, system_time in times:
    print(f"For {dr = } it took {system_time:2f}s")

For dr = 5 it took 654.591663s
For dr = 10 it took 430.043038s
For dr = 15 it took 373.674586s
For dr = 20 it took 355.669297s
